In [1]:
import pickle
import torch
import numpy as np
import IPython.display as ipd

In [2]:
from math import ceil
from librosa.filters import mel
from librosa.util import normalize
from melgan.interface import *

In [3]:
from factory.AutoVC_Adjust import AutoVC_Adjust

In [4]:
LEN_CROP = 176
DIM_NECK = 44
DIM_EMB = 256
D_MODEL = 80
FREQ = 22
BATCH_SIZE = 2
DEVICE = 'cpu'
ROOT = 'train_data_en_vctk_ver2'

In [5]:
org = AutoVC_Adjust(44,256,512,22).to(DEVICE)
org.load_state_dict(torch.load("model/autovc_adjust_en.pt", map_location=DEVICE))

<All keys matched successfully>

In [6]:
vocoder = MelVocoder(model_name='model/static/multi_speaker')

In [7]:
metadata = pickle.load(open(f'{ROOT}/train.pkl', "rb"))

In [72]:
source = 11
target = 29
sound = 3

In [73]:
metadata[source][0],metadata[target][0]

('p237', 'p256')

In [74]:
emb_org = np.expand_dims(metadata[source][1],axis=0)
emb_org.shape

(1, 256)

In [75]:
p_ = metadata[source][sound].replace("\\", "/")
uttr_org = np.load(f"{ROOT}/{p_}")[50:226]
uttr_org  = np.expand_dims(uttr_org ,axis=0)
uttr_org.shape

(1, 176, 80)

In [76]:
p_ = metadata[target][sound].replace("\\", "/")
uttr_trg = np.load(f"{ROOT}/{p_}")[50:226]
uttr_trg  = np.expand_dims(uttr_trg ,axis=0)
uttr_trg.shape

(1, 176, 80)

In [77]:
emb_trg =  np.expand_dims(metadata[target][1],axis=0)
emb_trg.shape

(1, 256)

## 原始聲音

In [78]:
audio_s = vocoder.inverse(torch.from_numpy(np.transpose(uttr_org,(0,2,1))))
ipd.Audio(audio_s,rate = 22050)

## 目標聲音

In [79]:
audio_t = vocoder.inverse(torch.from_numpy(np.transpose(uttr_trg,(0,2,1))))
ipd.Audio(audio_t,rate = 22050)

In [80]:
_,_,x_org,_ = org(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE),True,torch.from_numpy(uttr_trg).to(DEVICE))
uttr_org = x_org[0,:,:]

In [81]:
org_audio = vocoder.inverse(uttr_org.transpose(1,2))

## 轉換聲音

In [82]:
ipd.Audio(org_audio,rate = 22050)